# Monitor loan approval model

### Prerequisite

In order to execute this notebook you will need a deployed model. You can perform it with following [notebook](German%20credit%20risk%20prediction%20with%20Scikit%20for%20model%20monitoring.ipynb). Then you will need to copy `deployment_uid` to this notebook.

### Learning goals

In this notebook, you will learn how to:

- Retrieve existing deployment in Watson OpenScale
-  Make a WatsonOpenscale subscription
-  Prepare and run model monitoring and feedback logging
-  Run fairness monitor
-  Run drift monitor
-  Run custom monitors and metrics

### Contents

- [Setup](#setup)
- [OpenScale configuration](#openscale)
- [Quality monitor and feedback logging](#quality)
- [Fairness monitoring and explanations](#fairness)
- [Drift monitoring](#drift)
- [Custom monitors and metrics](#custom)
- [Summary](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

In [18]:
from dotenv import load_dotenv
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from ibm_watson_openscale import APIClient
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

import os
load_dotenv()

wx_apikey = os.getenv("WATSONX_APIKEY", None)
wx_region = os.getenv("IBM_CLOUD_URL", None)
# wx_projectid = os.getenv("PROJECT_ID", None)
space_id = os.getenv("WATSONX_SPACE_ID", None)
# deployment_uid = os.getenv("WATSONX_DEPLOYMENT_ID", None)
deployment_uid = "18a7a7a1-c8d9-40c6-8e10-8209aea18af7"
COS_API_KEY_ID = os.getenv("COS_API_KEY_ID", None)
COS_RESOURCE_CRN = os.getenv("COS_RESOURCE_CRN", None)
COS_ENDPOINT = os.getenv("COS_ENDPOINT", None)
IAM_AUTH_ENDPOINT = os.getenv("IAM_AUTH_ENDPOINT", None)
BUCKET_NAME = os.getenv("BUCKET_NAME", None)


In [19]:
DB_CREDENTIALS = None
SCHEMA_NAME = None
KEEP_MY_INTERNAL_POSTGRES = True
training_data_file_name = "dataset_loan_example_v2.csv"
MODEL_NAME = "loan auto ai deployment"
DEPLOYMENT_NAME = "loan auto ai deployment"
SERVICE_PROVIDER_NAME = "Watson Machine Learning V4"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook"

In [20]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "apikey": wx_apikey,
    "url": wx_region
}

client = APIClient(wml_credentials)
client.set.default_space(space_id)
# client.deployments.list()

deployment_url = client.deployments.get_details(deployment_uid)['entity']['status']['online_url']['url']
model_uid = client.deployments.get_details(deployment_uid)['entity']['asset']['id']

from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_openscale import APIClient

## Configure openscale
authenticator = IAMAuthenticator(apikey=wml_credentials["apikey"])
wos_client = APIClient(authenticator=authenticator)
wos_client.version

# wos_client.data_marts.show()
# wos_client.service_providers.show()
# wos_client.subscriptions.show()
# wos_client.data_sets.show()


'3.0.40'

# Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and set up a Python OpenScale client.

## Create schema and datamart

### Set up datamart

In [22]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print("Setting up external datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS["connection"]["postgres"]["hosts"][0]["hostname"],
                        username=DB_CREDENTIALS["connection"]["postgres"]["authentication"]["username"],
                        password=DB_CREDENTIALS["connection"]["postgres"]["authentication"]["password"],
                        db=DB_CREDENTIALS["connection"]["postgres"]["database"],
                        port=DB_CREDENTIALS["connection"]["postgres"]["hosts"][0]["port"],
                        ssl=True,
                        sslmode=DB_CREDENTIALS["connection"]["postgres"]["query_options"]["sslmode"],
                        certificate_base64=DB_CREDENTIALS["connection"]["postgres"]["certificate"]["certificate_base64"]
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print("Setting up internal datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print("Using existing datamart {}".format(data_mart_id))

Using existing datamart 9d0af833-251f-4678-a601-453456e8bb45


### Remove existing service provider connected with used  WML instance. 

Multiple service providers for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one. 

In [23]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

## Add service provider

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

**Note:** You can bind more than one engine instance if needed by calling `wos_client.service_providers.add` method. Next, you can refer to particular service provider using `service_provider_id`.

In [31]:
# Try this before below

# added_service_provider_result = wos_client.service_providers.add(
#         name=SERVICE_PROVIDER_NAME,
#         description=SERVICE_PROVIDER_DESCRIPTION,
#         service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
#         deployment_space_id = space_id,
#         operational_space_id = "production",
#         credentials=WMLCredentialsCloud(
#             apikey=wml_credentials["apikey"],
#             url=wml_credentials["url"],
#             instance_id=None
#         ),
#         background_mode=False
#     ).result
# service_provider_id = added_service_provider_result.metadata.id

# backup
service_provider_id = wos_client.service_providers.list().result.service_providers[0].metadata.id

## Subscriptions

### Remove existing credit risk subscriptions

This code removes previous subscriptions to the German Credit model to refresh the monitors with the new model and new data.

In [32]:
# subscriptions = wos_client.subscriptions.list().result.subscriptions
# for subscription in subscriptions:
#     sub_model_id = subscription.entity.asset.asset_id
#     if sub_model_id == model_uid:
#         wos_client.subscriptions.delete(subscription.metadata.id)
#         print("Deleted existing subscription for model", model_uid)

Following cells create the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [33]:
import ibm_boto3
from botocore.client import Config

cos_cli = ibm_boto3.client("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint=IAM_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)
cos_cli.download_file(BUCKET_NAME, training_data_file_name, training_data_file_name)

In [49]:
asset = Asset(
    asset_id=model_uid,
    url=deployment_url,
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION
)
asset_deployment = AssetDeploymentRequest(
    deployment_id=deployment_uid,
    name=DEPLOYMENT_NAME,
    deployment_type=DeploymentTypes.ONLINE,
    url=deployment_url
)
# First, set up the training data reference
training_data_reference = TrainingDataReference(
    type="cos",
    location=COSTrainingDataReferenceLocation(
        bucket=BUCKET_NAME,
        file_name=training_data_file_name
    ),
    connection=COSTrainingDataReferenceConnection.from_dict(
        {
            "resource_instance_id": COS_RESOURCE_CRN,
            "url": COS_ENDPOINT,
            "api_key": COS_API_KEY_ID,
            "iam_url": IAM_AUTH_ENDPOINT
        }
    )
)
# Now, set up the asset properties request
asset_properties_request = AssetPropertiesRequest(
    label_column="Y",
    probability_fields=["probability"],
    prediction_field="prediction",
    feature_fields=["Gender", "Married", "Dependents", "Education", "Self_Employed", "ApplicantIncome", "CoapplicantIncome", "LoanAmount", "Loan_Amount_Term", "Credit_History", "Property_Area"],
    categorical_fields=["Gender", "Married", "Dependents", "Education", "Self_Employed", "Property_Area"],
    training_data_reference=training_data_reference
)

In [50]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=asset,
        deployment=asset_deployment,
        asset_properties=asset_properties_request).result
subscription_id = subscription_details.metadata.id
print(subscription_details)

{
  "metadata": {
    "id": "a4338afe-2d7a-4bed-8932-10620efd77c5",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/98ff78eb326f477f8447f94142661697:9d0af833-251f-4678-a601-453456e8bb45:subscription:a4338afe-2d7a-4bed-8932-10620efd77c5",
    "url": "/v2/subscriptions/a4338afe-2d7a-4bed-8932-10620efd77c5",
    "created_at": "2024-09-30T14:21:21.855000Z",
    "created_by": "IBMid-6910005CCG"
  },
  "entity": {
    "data_mart_id": "9d0af833-251f-4678-a601-453456e8bb45",
    "service_provider_id": "c9abd68c-59db-47b1-b418-6930e61a0976",
    "asset": {
      "asset_id": "d41007a4-3f58-422c-adc5-13744670bc33",
      "url": "https://us-south.ml.cloud.ibm.com/ml/v4/deployments/18a7a7a1-c8d9-40c6-8e10-8209aea18af7/predictions",
      "asset_type": "model",
      "problem_type": "binary",
      "input_data_type": "structured"
    },
    "asset_properties": {
      "training_data_reference": {
        "secret_id": "6e6ec807-2713-4da7-8fa5-0a80536a5ced"
      },
      "label_column": "Y",

In [51]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

Payload data set id: ca34504a-beda-46ca-9438-1a2cada7be3d


### Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. The sends a few records for predictions.

In [52]:
fields = [
                                "Gender",
                                "Married",
                                "Dependents",
                                "Education",
                                "Self_Employed",
                                "ApplicantIncome",
                                "CoapplicantIncome",
                                "LoanAmount",
                                "Loan_Amount_Term",
                                "Credit_History",
                                "Property_Area"
                        ]
values = [
  ["Male", "Yes", "0", "Graduate", "No", 5000, 2000, 150, 360, 1, "Urban"]
]

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
predictions = client.deployments.score(deployment_uid, payload_scoring)

print("Single record scoring result:", "\n fields:", predictions["predictions"][0]["fields"], "\n values: ", predictions["predictions"][0]["values"][0])

Single record scoring result: 
 fields: ['prediction', 'probability'] 
 values:  ['Y', [0.18525838093324143, 0.8147416190667586]]


In [55]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response=predictions,
                   response_time=460
               )])
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

Number of records in the payload logging table: 1


# Quality monitoring and feedback logging <a name="quality"></a>

## Enable quality monitoring

The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 70%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [56]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters
).result




 Waiting for end of monitor instance creation 12b90cf2-e514-471f-a70c-bd4eb585cadd 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [57]:
quality_monitor_instance_id = quality_monitor_details.metadata.id

## Feedback logging

The code below downloads and stores enough feedback data to meet the minimum threshold so that OpenScale can calculate a new accuracy measurement. It then kicks off the accuracy monitor. The monitors run hourly, or can be initiated via the Python API, the REST API, or the graphical user interface.

In [58]:
# !rm additional_feedback_data_v2.json
# !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/additional_feedback_data_v2.json

### Get feedback logging dataset ID

In [59]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

{
  "data_sets": [
    {
      "metadata": {
        "id": "f2d1fb1f-83d5-4fd4-994d-b08ece32bac5",
        "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/181ed6cc388f47bd9d862fe066f9cfce:9d0af833-251f-4678-a601-453456e8bb45:data_set:f2d1fb1f-83d5-4fd4-994d-b08ece32bac5",
        "url": "/v2/data_sets/f2d1fb1f-83d5-4fd4-994d-b08ece32bac5",
        "created_at": "2024-09-30T14:22:44.269000Z",
        "created_by": "iam-ServiceId-2e5c9fda-38bf-4279-9712-cdb3b6f3a7ad",
        "modified_at": "2024-09-30T14:22:44.802000Z",
        "modified_by": "iam-ServiceId-2e5c9fda-38bf-4279-9712-cdb3b6f3a7ad"
      },
      "entity": {
        "data_mart_id": "9d0af833-251f-4678-a601-453456e8bb45",
        "name": "a4338afe-2d7a-4bed-8932-10620efd77c5_feedback",
        "description": "a4338afe-2d7a-4bed-8932-10620efd77c5_feedback",
        "type": "feedback",
        "target": {
          "target_type": "subscription",
          "target_id": "a4338afe-2d7a-4bed-8932-10620efd77c5"
        },
    

In [61]:
# import json
# with open('additional_feedback_data_v2.json') as feedback_file:
#     additional_feedback_data = json.load(feedback_file)

In [72]:
import random

def generate_record():
    return {
        # "Loan_ID": f"LP{random.randint(100000, 999999)}",
        "Gender": random.choice(["Male", "Female"]),
        "Married": random.choice(["Yes", "No"]),
        "Dependents": random.choice(["0", "1", "2", "3+"]),
        "Education": random.choice(["Graduate", "Not Graduate"]),
        "Self_Employed": random.choice(["Yes", "No"]),
        "ApplicantIncome": random.randint(1500, 20000),
        "CoapplicantIncome": random.randint(0, 15000),
        "LoanAmount": random.randint(9, 700) * 1000,
        "Loan_Amount_Term": random.choice([12, 36, 60, 84, 120, 180, 240, 300, 360, 480]),
        "Credit_History": random.choice([0, 1]),
        "Property_Area": random.choice(["Urban", "Semiurban", "Rural"]),
        "Loan_Status": random.choice(["Y", "N"])
    }


def generate_record_2():
    return [
        # f"LP{random.randint(100000, 999999)}",
        random.choice(["Male", "Female"]),
        random.choice(["Yes", "No"]),
        random.choice(["0", "1", "2", "3+"]),
        random.choice(["Graduate", "Not Graduate"]),
        random.choice(["Yes", "No"]),
        random.randint(1500, 20000),
        random.randint(0, 15000),
        random.randint(9, 700) * 1000,
        random.choice([12, 36, 60, 84, 120, 180, 240, 300, 360, 480]),
        random.choice([0, 1]),
        random.choice(["Urban", "Semiurban", "Rural"]),
        # "Loan_Status": random.choice(["Y", "N"])
]

feedback_data = [generate_record() for _ in range(120)]
feedback_data_2 = [generate_record_2() for _ in range(60)]
# Print the first few records as an example
for record in feedback_data_2[:5]:
    print(record)

['Male', 'Yes', '0', 'Graduate', 'No', 13525, 2515, 255000, 60, 0, 'Rural']
['Female', 'No', '2', 'Graduate', 'Yes', 12039, 3347, 570000, 84, 0, 'Rural']
['Female', 'No', '0', 'Graduate', 'Yes', 15327, 3060, 686000, 120, 1, 'Semiurban']
['Female', 'Yes', '3+', 'Graduate', 'No', 17703, 4259, 490000, 12, 1, 'Urban']
['Female', 'Yes', '3+', 'Not Graduate', 'No', 16169, 12201, 679000, 12, 0, 'Semiurban']


In [73]:
wos_client.data_sets.store_records(feedback_dataset_id, request_body=feedback_data, background_mode=False)




 Waiting for end of storing records with request id: 764d2397-6c7e-406f-b1b2-ccf64b005d18 




pending
active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [74]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

240

In [75]:
wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id)

1

In [76]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 1d2e94ca-24d1-43f1-9ab6-40ac6eac5564 




finished

---------------------------
 Successfully finished run 
---------------------------




In [77]:
result_metrics = wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2024-09-30 14:27:26.572000+00:00,area_under_roc,d1be4a74-0d49-4a7e-a506-1f8b2f9d93e1,0.0,0.8,None,['model_type:original'],quality,12b90cf2-e514-471f-a70c-bd4eb585cadd,1d2e94ca-24d1-43f1-9ab6-40ac6eac5564,subscription,a4338afe-2d7a-4bed-8932-10620efd77c5
2024-09-30 14:27:26.572000+00:00,matthews_correlation_coefficient,d1be4a74-0d49-4a7e-a506-1f8b2f9d93e1,0.0,0.8,None,['model_type:original'],quality,12b90cf2-e514-471f-a70c-bd4eb585cadd,1d2e94ca-24d1-43f1-9ab6-40ac6eac5564,subscription,a4338afe-2d7a-4bed-8932-10620efd77c5
2024-09-30 14:27:26.572000+00:00,accuracy,d1be4a74-0d49-4a7e-a506-1f8b2f9d93e1,0.6541666666666667,0.8,None,['model_type:original'],quality,12b90cf2-e514-471f-a70c-bd4eb585cadd,1d2e94ca-24d1-43f1-9ab6-40ac6eac5564,subscription,a4338afe-2d7a-4bed-8932-10620efd77c5
2024-09-30 14:27:26.572000+00:00,label_skew,d1be4a74-0d49-4a7e-a506-1f8b2f9d93e1,0.0,-0.5,0.5,['model_type:original'],quality,12b90cf2-e514-471f-a70c-bd4eb585cadd,1d2e94ca-24d1-43f1-9ab6-40ac6eac5564,subscription,a4338afe-2d7a-4bed-8932-10620efd77c5
2024-09-30 14:27:26.572000+00:00,gini_coefficient,d1be4a74-0d49-4a7e-a506-1f8b2f9d93e1,-1.0,0.8,None,['model_type:original'],quality,12b90cf2-e514-471f-a70c-bd4eb585cadd,1d2e94ca-24d1-43f1-9ab6-40ac6eac5564,subscription,a4338afe-2d7a-4bed-8932-10620efd77c5
2024-09-30 14:27:26.572000+00:00,log_loss,d1be4a74-0d49-4a7e-a506-1f8b2f9d93e1,34.53877639491088,None,0.8,['model_type:original'],quality,12b90cf2-e514-471f-a70c-bd4eb585cadd,1d2e94ca-24d1-43f1-9ab6-40ac6eac5564,subscription,a4338afe-2d7a-4bed-8932-10620efd77c5
2024-09-30 14:27:26.572000+00:00,area_under_pr,d1be4a74-0d49-4a7e-a506-1f8b2f9d93e1,0.0,0.8,None,['model_type:original'],quality,12b90cf2-e514-471f-a70c-bd4eb585cadd,1d2e94ca-24d1-43f1-9ab6-40ac6eac5564,subscription,a4338afe-2d7a-4bed-8932-10620efd77c5
2024-09-30 14:27:26.572000+00:00,brier_score,d1be4a74-0d49-4a7e-a506-1f8b2f9d93e1,0.0,None,0.8,['model_type:original'],quality,12b90cf2-e514-471f-a70c-bd4eb585cadd,1d2e94ca-24d1-43f1-9ab6-40ac6eac5564,subscription,a4338afe-2d7a-4bed-8932-10620efd77c5
2024-09-30 14:25:15.760000+00:00,area_under_roc,4ddbddfe-2a8a-423f-a216-a7660e74e5c2,0.0,0.8,None,['model_type:original'],quality,12b90cf2-e514-471f-a70c-bd4eb585cadd,2072d216-feca-42ce-8a2b-0db9c5012b07,subscription,a4338afe-2d7a-4bed-8932-10620efd77c5
2024-09-30 14:25:15.760000+00:00,matthews_correlation_coefficient,4ddbddfe-2a8a-423f-a216-a7660e74e5c2,0.0,0.8,None,['model_type:original'],quality,12b90cf2-e514-471f-a70c-bd4eb585cadd,2072d216-feca-42ce-8a2b-0db9c5012b07,subscription,a4338afe-2d7a-4bed-8932-10620efd77c5


Note: First 10 records were displayed.


In [78]:
result_metrics

# Fairness monitoring and explanations <a name="fairness"></a>

### Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, Sex and Age. In each case, we must specify:
  * Which model feature to monitor
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 200 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [79]:
wos_client.monitor_instances.show()

9d0af833-251f-4678-a601-453456e8bb45,active,a4338afe-2d7a-4bed-8932-10620efd77c5,subscription,quality,2024-09-30 14:22:42.862000+00:00,12b90cf2-e514-471f-a70c-bd4eb585cadd
9d0af833-251f-4678-a601-453456e8bb45,active,a4338afe-2d7a-4bed-8932-10620efd77c5,subscription,mrm,2024-09-30 14:21:54.465000+00:00,6e25d05d-e346-47ac-b0c9-ec681bb778e4
9d0af833-251f-4678-a601-453456e8bb45,active,a4338afe-2d7a-4bed-8932-10620efd77c5,subscription,performance,2024-09-30 14:21:24.863000+00:00,f98cec6e-5290-4d38-82f7-f4254ee9c787
9d0af833-251f-4678-a601-453456e8bb45,active,e020efd1-2e3e-4755-b2ea-6d1d939cd7fa,subscription,performance,2024-09-30 14:18:30.122000+00:00,5ec5d684-6e8d-443b-9d4d-4f328f2b1408
9d0af833-251f-4678-a601-453456e8bb45,active,91af289a-0e4c-453a-ba05-b7bd4653baa7,subscription,performance,2024-09-30 14:17:24.256000+00:00,ae69da17-a3ca-4c07-b339-f5786af0c328
9d0af833-251f-4678-a601-453456e8bb45,active,cf5a33f1-09c6-4d4c-8cfa-26799a6eee92,subscription,mrm,2024-09-12 18:01:36.876000+00:00,5080e7e8-fab7-4428-907f-dcba5c25d2ec
9d0af833-251f-4678-a601-453456e8bb45,active,cf5a33f1-09c6-4d4c-8cfa-26799a6eee92,subscription,model_health,2024-09-12 18:01:36.175000+00:00,68e7f2fc-a453-4a30-8360-1b17cf56bbf2
9d0af833-251f-4678-a601-453456e8bb45,active,838b600d-06b6-4629-9fdc-a0c1a2f3df6b,subscription,model_health,2024-09-12 14:50:56.504000+00:00,601c8ee8-8b68-4ff6-8f78-0dc904c3cf92
9d0af833-251f-4678-a601-453456e8bb45,active,838b600d-06b6-4629-9fdc-a0c1a2f3df6b,subscription,mrm,2024-09-12 14:52:10.649000+00:00,d4019cae-28bd-432b-a3a2-c6bed8c16604
9d0af833-251f-4678-a601-453456e8bb45,active,838b600d-06b6-4629-9fdc-a0c1a2f3df6b,subscription,fairness,2024-09-12 14:51:49.188000+00:00,5e77cbd6-f3ed-4962-9881-a8099d518ad6


Note: First 10 records were displayed.


In [80]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Gender",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        # {"feature": "Age",
        #  "majority": [[26, 75]],
        #  "minority": [[18, 25]],
        #  "threshold": 0.95
        #  }
    ],
    "favourable_class": ["N"],
    "unfavourable_class": ["Y"],
    "min_records": 4
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id




 Waiting for end of monitor instance creation 9538dd2e-e09a-4566-ab1a-3c253a799668 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Drift configuration

In [81]:
from ibm_watson_openscale.supporting_classes import Target

# # First, try to delete the existing monitor
# existing_monitor_id = "90bda763-3a37-42d2-9c89-0e4ebad21c46"
# try:
#     wos_client.monitor_instances.delete(existing_monitor_id)
#     print(f"Existing monitor {existing_monitor_id} deleted successfully.")
# except Exception as e:
#     print(f"Error deleting existing monitor: {str(e)}")

# Now, create a new drift monitor
feature_list = [
    "Gender", "Married", "Dependents", "Education", "Self_Employed",
    "ApplicantIncome", "CoapplicantIncome", "LoanAmount", "Loan_Amount_Term",
    "Credit_History", "Property_Area"
]

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "features": feature_list,
    "min_samples": 10,
    "drift_threshold": 0.1,
    "train_drift_model": True,
    "enable_model_drift": True,
    "enable_data_drift": True
}

try:
    drift_monitor_details = wos_client.monitor_instances.create(
        data_mart_id=data_mart_id,
        background_mode=False,
        monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
        target=target,
        parameters=parameters
    ).result
    
    drift_monitor_instance_id = drift_monitor_details.metadata.id
    print(f"New drift monitor created successfully. Instance ID: {drift_monitor_instance_id}")
except Exception as e:
    print(f"Error creating new drift monitor: {str(e)}")
    import traceback
    print(traceback.format_exc())




 Waiting for end of monitor instance creation 99b6f452-671f-4fa7-887c-f02893a2097e 




preparing.........
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------


New drift monitor created successfully. Instance ID: 99b6f452-671f-4fa7-887c-f02893a2097e


## Score the model again now that monitoring is configured

This next section randomly selects 200 records from the data feed and sends those records to the model for predictions. This is enough to exceed the minimum threshold for records set in the previous section, which allows OpenScale to begin calculating fairness.

In [86]:
# import random

# with open('german_credit_feed.json', 'r') as scoring_file:
#     scoring_data = json.load(scoring_file)

# fields = scoring_data['fields']
# values = []
# for _ in range(200):
#     values.append(random.choice(scoring_data['values']))

# Define the fields
fields = [
    "Gender", "Married", "Dependents", "Education", "Self_Employed",
    "ApplicantIncome", "CoapplicantIncome", "LoanAmount", "Loan_Amount_Term",
    "Credit_History", "Property_Area"
]

# Create the final structure
payload_scoring = {
    "input_data": [
        {
            "fields": fields,
            "values": feedback_data_2
        }
    ]
}
# payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

scoring_response = client.deployments.score(deployment_uid, payload_scoring)
time.sleep(5)

if pl_records_count == 8:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response=scoring_response,
                   response_time=460
               )])
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

# wos_client.data_sets.store_records(feedback_dataset_id, request_body=feedback_data, background_mode=False)

In [87]:
print("Number of records in payload table:", wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id))

Number of records in payload table: 61


## Run fairness monitor

Kick off a fairness monitor run on current data. The monitor runs hourly, but can be manually initiated using the Python client, the REST API, or the graphical user interface.

In [88]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 89508181-28f6-4861-91e4-6186ba8de448 




running
finished

---------------------------
 Successfully finished run 
---------------------------




In [89]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2024-09-30 14:29:39.366866+00:00,fairness_value,1c9e6d5f-82bd-4312-90a2-f58eab674731,100.0,80.0,None,"['feature:Gender', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,9538dd2e-e09a-4566-ab1a-3c253a799668,89508181-28f6-4861-91e4-6186ba8de448,subscription,a4338afe-2d7a-4bed-8932-10620efd77c5


## Configure Explainability

Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [92]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation 591e193d-b96c-4b0c-92ea-3ae05ea2ae07 




preparing....
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Run explanation for sample record

In [93]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)


Running explanations on scoring IDs: ['MRM_a664d387-a05e-4e5e-93d1-6913cb0fcea0-1-1']


ApiException: Error: The subscription id should be provided in the request., Code: 400

# Custom monitors and metrics <a name="custom"></a>

## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [ ]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id, limit=5)

,252000,405806f4c4d619603a2f3a1596784e3e-1,2024-09-22T13:49:05.215385Z,LP684261,0.7233383103328592,Y,1,Rural,Yes,14887,36,ba55abdb-33ef-4d15-a92f-2902def1ea7b,5694,Not Graduate,Yes,Male,2,"[0.2766616896671408, 0.7233383103328592]"
,634000,405806f4c4d619603a2f3a1596784e3e-10,2024-09-22T13:49:05.215385Z,LP405309,0.6838387382381103,Y,1,Rural,No,4789,360,ba55abdb-33ef-4d15-a92f-2902def1ea7b,1203,Graduate,No,Female,3+,"[0.31616126176188974, 0.6838387382381103]"
,240000,405806f4c4d619603a2f3a1596784e3e-11,2024-09-22T13:49:05.215385Z,LP397657,0.5498042528462761,N,0,Urban,Yes,15827,120,ba55abdb-33ef-4d15-a92f-2902def1ea7b,1463,Not Graduate,No,Male,3+,"[0.5498042528462761, 0.45019574715372396]"
,275000,405806f4c4d619603a2f3a1596784e3e-12,2024-09-22T13:49:05.215385Z,LP478590,0.5996403308475718,Y,1,Urban,Yes,17249,480,ba55abdb-33ef-4d15-a92f-2902def1ea7b,7290,Not Graduate,Yes,Female,0,"[0.40035966915242815, 0.5996403308475718]"
,67000,405806f4c4d619603a2f3a1596784e3e-13,2024-09-22T13:49:05.215385Z,LP239087,0.6310522662585273,Y,0,Urban,Yes,6664,240,ba55abdb-33ef-4d15-a92f-2902def1ea7b,10931,Not Graduate,Yes,Male,1,"[0.3689477337414727, 0.6310522662585273]"


<a id="summary"></a>
##  Summary and next steps     

You successfully completed this notebook! 
 
You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.


OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and can be deleted if necessary. You can use this notebooks cells to delete deployments.


Check out our <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener noreferrer">Online Documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts. 

## Authors

Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Szymon Kucharczyk, Software Engineer at IBM Watson Machine Learning.

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.